# TESTS
* BELOW I HAVE TESTED IF TRANSFORMS CALCULATE THE RANDOM PROBABILITY SEPERATELY FOR EVERY IMAGE IN A
* ANSWER -> IT DOES NOT

In [ ]:
t = T.RandomResizedCrop((image_size, image_size))
import torchvision
img = torchvision.io.read_image("/home/kuartis-dgx1/utku/UniVIP/dataset/211110_sereflikochisar__dfas__3_rgb--image_on_00006383.png")
img = img.unsqueeze(dim=0).broadcast_to((3,*img.shape))
print(img.shape)
params= t.get_params(img, scale=(0.08, 1.0), ratio=(0.75, 1.3333333333333333)) # returns top: int, left: int, height: int, width
print(params)

In [ ]:
import torchvision
img = torchvision.io.read_image("/home/kuartis-dgx1/utku/UniVIP/dataset/211110_sereflikochisar__dfas__3_rgb--image_on_00006383.png")
img = img.unsqueeze(dim=0).broadcast_to((3,*img.shape))
print(img.shape)
t_color = T.ColorJitter(0.8, 0.8, 0.8, 0.2)
params = t_color.get_params((0.2, 1.8), (0.2, 1.8), (0.2, 1.8), (-0.2, 0.2))
print(params)

In [ ]:
import torchvision
import torchvision.transforms.functional as F
class RandomHorizontalFlip(torch.nn.Module):
    def __init__(self, p=0.5):
        super().__init__()
        self.p = p

    def forward(self, img):
        prob = torch.rand(1)
        print(prob)
        if prob < self.p:
            return F.hflip(img)
        return img
img = torchvision.io.read_image("/home/kuartis-dgx1/utku/UniVIP/dataset/211110_sereflikochisar__dfas__3_rgb--image_on_00006383.png")
img = img.unsqueeze(dim=0).broadcast_to((3,*img.shape))
print(img.shape)
a = RandomHorizontalFlip()(img)

In [ ]:
import torchvision.transforms.functional as F
print(img.shape)
_, height, width = F.get_dimensions(img)

In [ ]:
import torchvision.transforms.functional as F
def random_hflip(img, p):
    flip_bool = torch.rand(1) < p
    return (F.hflip(img),flip_bool) if flip_bool else (img, flip_bool)

In [ ]:
import torch.nn.functional as F
conv = torch.nn.Conv2d(3, 4, 2)

at = torch.ones((3,4,3,60,60))
print(at.size())
bt = at.reshape(-1,*at.size()[-3:])
print(bt.size())
conv(bt).shape
instance = F.interpolate(bt, size=(96, 96), mode="bicubic") # resize instance to 96x96
instance.shape
torch.stack([bt,bt]).shape


## The Cost matrix

In [ ]:
import torch

# Assuming O and T are your input matrices of size KxM and KxN respectively.
# Horizontal stack of column vectors of lenght K (K is number of instances)
O_matrix = torch.tensor([[3,5,6],[4,12,8]], dtype=torch.float)# (M is number of features) 
T_matrix = torch.tensor([[1,2,3],[4,2,5]], dtype=torch.float)# (N is number of features)

# Step 1: Compute DotProductMatrix
DotProductMatrix = torch.mm(O_matrix.t(), T_matrix)

# Step 2: Compute Norm matrices
NormMatrixO = torch.norm(O_matrix, dim=0, keepdim=True)
NormMatrixT = torch.norm(T_matrix, dim=0, keepdim=True)

# Step 3: Compute C
C = 1 - (DotProductMatrix / (NormMatrixO.t() * NormMatrixT))


In [ ]:
import torch

# Assuming O and T are your input matrices of size KxM and KxN respectively.
# Horizontal stack of column vectors of lenght K (K is number of instances)
O = torch.tensor([[3,5,6],[4,12,8]], dtype=torch.float)# (M is number of features) 
T = torch.tensor([[1,2,3],[4,2,5]], dtype=torch.float)# (N is number of features)

# Step 1: Compute DotProductMatrix
DotProductMatrix = torch.mm(O.t(), T)
O.shape

In [ ]:
# Step 2: Compute Norm matrices, output will be a horizontal vector.

NormMatrixO = torch.norm(O, dim=0, keepdim=True) # find norm of columns
NormMatrixT = torch.norm(T, dim=0, keepdim=True)
NormMatrixO.t()

In [ ]:
NormMatrixT

In [ ]:

# TODO check if this (NormMatrixO.t() * NormMatrixT) matches the target indices.
# Step 3: Compute C
C = 1 - (DotProductMatrix / (NormMatrixO.t() * NormMatrixT))


In [ ]:
C

# test concatentation   

In [ ]:
import torch
import torch.nn.functional as F
import torchvision.ops

def get_concatenated_instances(img, overlapping_boxes):
    # img shape should be (b, c, h, w) - batch, channels, height, width
    # overlapping_boxes shape should be (b, n, 4) - batch, number of boxes, coordinates (x1, y1, x2, y2)

    # Number of boxes per image
    num_boxes = overlapping_boxes.size(1)
    
    # Create batch indices to be concatenated with boxes -> (batch_size*K), each box will have an index (showing where it belongs)
    batch_indices = torch.arange(img.size(0), dtype=torch.float32).view(-1, 1).repeat(1, num_boxes).view(-1, 1)
    print(batch_indices.shape)
    print(batch_indices)
    
    # Reshape boxes for roi_align
    boxes = overlapping_boxes.view(-1, 4) # Collect total number of boxes in the first dim (batch_size*K)
    
    print(boxes.shape)
    # Concatenate batch indices with boxes, index shows which image in a batch each box belongs
    boxes_with_indices = torch.cat([batch_indices, boxes], dim=1)
    
    # Crop and resize using roi_align
    output_size = (96, 96)
    instances = torchvision.ops.roi_align(img, boxes_with_indices, output_size)
    
    # Now instances tensor has shape (b * n, c, 96, 96), TODO you can reshape it if needed
    return instances

# Example usage
b, c, h, w = 3, 3, 256, 256
img_batch = torch.randn(b, c, h, w)
overlapping_boxes_batch = torch.tensor([[[50, 50, 100, 100], [30, 30, 80, 80]], [[10, 10, 60, 60], [20, 20, 70, 70]], [[40, 40, 110, 110], [50, 50, 120, 120]]])

# instances_batch = get_concatenated_instances(img_batch, overlapping_boxes_batch)
# print(instances_batch.shape)  # Should be (b * n, c, 96, 96)
img = torch.randn(1, c, h, w)
overlapping_boxes = torch.tensor([[[50, 50, 100, 100], [30, 30, 80, 80]]])
instances = get_concatenated_instances(img, overlapping_boxes)
print(instances.shape)  # Should be (b * n, c, 96, 96)


In [ ]:
print([*overlapping_boxes.shape[0:2], *instances.shape[-3:]])
instances = instances.reshape(*overlapping_boxes.shape[0:2], *instances.shape[-3:])
torch.concat(instances, dim=1)

In [ ]:
O_matrix = torch.randn(3, 4, 5)  # (batch_size, instance numbers K, number of features)
T_matrix = torch.randn(3, 4, 5)  # (batch_size, instance numbers K, number of features)
dot_product_matrix = torch.matmul(O_matrix, T_matrix.transpose(1, 2))  # (batch_size, number of features, number of features)
norm_vector_O = torch.norm(O_matrix, dim=-1, keepdim=True) # normalize over features
norm_vector_T = torch.norm(T_matrix, dim=-1, keepdim=True) # normalize over features
print(norm_vector_O.shape)
print(norm_vector_T.shape)
torch.matmul(norm_vector_O, norm_vector_T.transpose(1, 2)).shape


# TEST SINKHORN DISTANCE

In [ ]:
# Step 1: Compute dot_product_matrix
online_pred_avg = torch.randn(3, 1, 5)  # (batch_size, instance numbers K, number of features)
O_matrix = torch.randn(3, 4, 5)  # (batch_size, instance numbers K, number of features)
target_proj_avg = torch.randn(3, 1, 5)  # (batch_size, instance numbers K, number of features)
T_matrix = torch.randn(3, 4, 5)  # (batch_size, instance numbers K, number of features)
dot_product_matrix = torch.matmul(O_matrix, T_matrix.transpose(1, 2))  # (batch_size, instance numbers K, instance numbers K)
# Step 2: Compute Norm matrices
norm_vector_O = torch.norm(O_matrix, dim=-1, keepdim=True) # normalize over features
norm_vector_T = torch.norm(T_matrix, dim=-1, keepdim=True) # normalize over features
# Step 3: Compute C
ot_cosine_similarity_matrix = (dot_product_matrix / torch.matmul(norm_vector_O, norm_vector_T.transpose(1, 2)))
cost_matrix = 1 - ot_cosine_similarity_matrix
a_vector = torch.nn.functional.relu(torch.matmul(T_matrix, online_pred_avg.transpose(1, 2))) # (batch_size, instance numbers K, 1)
print(a_vector.values.shape)
print(torch.matmul(O_matrix, target_proj_avg.transpose(1, 2)).shape)
b_vector = torch.nn.functional.relu(torch.matmul(O_matrix, target_proj_avg.transpose(1, 2))) # (batch_size, instance numbers K, 1)
print(b_vector.shape)
print(b_vector)


In [ ]:
at = torch.randn(1, 4, 1)  # (batch_size, instance numbers K, number of features)
at.squeeze(dim=-1).shape # demander a (batch_size, instance numbers K, 1)

In [ ]:
# (batch_size, instance numbers K, instance numbers K)
optimal_plan_matrix = torch.randn(5, 4, 4)  # (batch_size, instance numbers K, number of features)
ot_cosine_similarity_matrix = torch.randn(5, 4, 4)  # (batch_size, instance numbers K, number of features)
torch.sum(-torch.mul(optimal_plan_matrix,ot_cosine_similarity_matrix), dim=(-2,-1)).mean() # Forces similar instance representations to be close to each other.


In [ ]:
import torch
image_size = 224
torch.randn(2, 3, image_size, image_size, device=0).shape

In [ ]:
import torchvision
from torch import nn
import random
img = torchvision.io.read_image("/home/kuartis-dgx1/utku/UniVIP/data_ops/211110_sereflikochisar__dfas__3_rgb--image_on_00006383.png")
print(img.dtype)
img[0,0,0] = 255
torchvision.transforms.functional.adjust_brightness(img, brightness_factor=0.2)
torchvision.transforms.functional.adjust_contrast(img, 0.2)
torchvision.transforms.functional.adjust_saturation(img, 0.2)

class RandomApply(nn.Module):
    def __init__(self, fn, p):
        super().__init__()
        self.fn = fn
        self.p = p
    def forward(self, x):
        return x if random.random() > self.p else self.fn(x)

from torchvision import transforms as T
import torch
img = torch.rand(size=(1, 3, 224, 224))*255
# img = img.unsqueeze(dim=0).broadcast_to((3,*img.shape))
print(img.dtype)
print((img).max())
col_jit = RandomApply(T.ColorJitter(0.4, 0.4, 0.2, 0.1), p = 0.8)
print(col_jit(img).max())
print(T.ColorJitter(0.4, 0.4, 0.2, 0.1)(img).min())

In [ ]:
torchvision.transforms.functional.adjust_brightness(img/255, brightness_factor=0.2)
(img/255).dtype


In [ ]:
torch.rand(size=(3, 1, 3, 255, 255)).max()

In [ ]:
import cv2
im = cv2.imread("/home/kuartis-dgx1/utku/UniVIP/data_ops/211110_sereflikochisar__dfas__3_rgb--image_on_00006383.png")
print(im)

In [ ]:
img = torchvision.io.read_image("/home/kuartis-dgx1/utku/UniVIP/data_ops/211110_sereflikochisar__dfas__3_rgb--image_on_00006383.png")
torchvision.transforms.functional.adjust_brightness(img, brightness_factor=0.2)


# CHECK NUMBER OF SMALL IMAGES

In [ ]:
from pathlib import Path
import cv2
import multiprocessing

def process_image(file_path):
    # Check if the file is an image (you can add more image extensions if necessary)
    if file_path.suffix.lower() in ['.jpg', '.jpeg', '.png', '.bmp']:
        # Read the image using OpenCV
        image = cv2.imread(str(file_path))
        
        # Calculate the size of the smaller edge
        height, width, _ = image.shape
        smaller_edge = min(height, width)
        
        return smaller_edge < 64, file_path
    else:
        return False, False

if __name__ == '__main__':
    # Specify the path to the folder containing the images
    folder_path = Path('/raid/utku/datasets/COCO_dataset/train2017/')
    
    # Initialize counters for images with edges smaller than 64 and 70
    count_smaller_than_64 = 0

    # Create a pool of worker processes
    pool = multiprocessing.Pool(80)

    # Iterate through each file in the folder
    results = pool.map(process_image, folder_path.glob('*'))

    # Process the results
    smaller_files =[]
    for smaller_than_64, file_path in results:
        if smaller_than_64:
            count_smaller_than_64 += 1
            smaller_files.append(file_path)

    # Print the results
    print(f"Number of images with edge smaller than 64: {count_smaller_than_64}")
    print(f"smaller_files are {smaller_files}")


In [ ]:
import torch
torch.tensor([1]).to(7).device

# CHECK TEST CODE

In [ ]:
# Load the state_dict from the .pt file
from torchvision import models
from uni_vip import UVIP
from torch.nn.parallel import DistributedDataParallel as DDP
import torch
import torch.distributed as dist
import os
from typing import OrderedDict

rank = 0
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12356' # set port for communication
dist.init_process_group(backend='nccl',rank=0, world_size=1) # For multi GPU train.

# If you're in a distributed environment, make sure all processes are synchronized before loading
state_dict = torch.load("/home/kuartis-dgx1/utku/UniVIP/train/uni_vip_pretrained_model.pt", map_location=lambda storage, loc: storage.cuda(0))
# state_dict = model.state_dict()
new_state_dict = {}

module_key_prefix = "module.online_encoder.net."
for name, param in state_dict.items():
    if name.startswith(module_key_prefix):
        new_state_dict[name.replace(module_key_prefix, "")] = param

new_linear_layer = torch.nn.Linear(2048, 1000)
new_state_dict["fc.weight"] = new_linear_layer.weight
new_state_dict["fc.bias"] = new_linear_layer.bias

new_state_dict = OrderedDict(new_state_dict)
resnet = models.resnet50(pretrained=False)
# Assign the updated state_dict to the mode
print(state_dict.keys())
print(new_state_dict.keys())
print(resnet.state_dict().keys())

In [ ]:
resnet.load_state_dict(new_state_dict)

# Freeze previous layers
for param in resnet.parameters():
    param.requires_grad = False

# Enable training for the new linear layer
for param in resnet.fc.parameters():
    param.requires_grad = True

# model is ready
# TODO 
# NOTE To let a non-DDP model load a state dict from a DDP model, consume_prefix_in_state_dict_if_present() needs to be applied to strip the prefix “module.” in the DDP state dict before loading.
resnet = resnet.to(rank)
resnet = DDP(resnet, device_ids=[rank])
resnet = torch.nn.SyncBatchNorm.convert_sync_batchnorm(resnet) # Not sure if UniVIP does this.


load whole univip

In [ ]:
# Load the state dictionary into the model
model.load_state_dict({k: state_dict[k] for k in state_dict if k in model.state_dict()})

# Optional: If you're in a distributed environment, make sure all processes are synchronized after loading
if torch.distributed.is_initialized():
    torch.distributed.barrier()

state_dict = model.state_dict()
for name, param in state_dict.items():
    if name.startswith("module.online_encoder"):
        state_dict[name.replace("module.online_encoder.", "")] = param

load only resnet

In [ ]:
from typing import OrderedDict
state_dict = model.state_dict()
new_state_dict = {}
for name, param in state_dict.items():
    if name.startswith("module.online_encoder"):
        new_state_dict[name.replace("module.online_encoder.net.", "")] = param
new_state_dict = OrderedDict(new_state_dict)


In [ ]:
new_state_dict

In [ ]:
resnet = models.resnet50(pretrained=False)
print(mod.state_dict().keys())
print(new_state_dict.keys())
